In [121]:
# Dependencies
import pandas as pd
import os

In [122]:
# Save csv and set to path
loadingSchoolData = "Resources/schools_complete.csv"
loadingStudentData = "Resources/students_complete.csv"

In [123]:
#Read csv file and save as DataFrame
schoolData = pd.read_csv(loadingSchoolData)
studentData = pd.read_csv(loadingStudentData)

In [124]:
# Create one data set
fullSchoolData = pd.merge(schoolData, studentData, how="left", on=["school_name", "school_name"])
fullSchoolData.head()

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


# District Summary

* The lowest spending per student tended to have the better over passing rate. The schools that spend the most per student, had the lowest overall passing percentage.

* Charter schools had higher scoress across the board. Charter schools took the top 5 schools in PyCity where the disctict level schools were the bottom 5 schools in PyCity.

* With the excewption of Wilson High School, a charter school; the rest of the schools that were considered a large school (2,000-5,000 students); had overall passing scores less than 60%.

In [125]:
# Calculate total number of unique schools
schoolCountUnique = fullSchoolData["school_name"].unique()
schoolCount = len(schoolCountUnique)
schoolCount

15

In [126]:
# calculcate the total number of studnets
studentCount = fullSchoolData["Student ID"].count()
studentCount

39170

In [127]:
# calculate total budget
totalBudget = schoolData["budget"].sum()
totalBudget

24649428

In [128]:
# Calcuclate average math score
averageMathScore = fullSchoolData["math_score"].mean()
averageMathScore

78.98537145774827

In [129]:
# Calculcate average reading score
averageReadingScore = fullSchoolData["reading_score"].mean()
averageReadingScore

81.87784018381414

In [130]:
# Calculate % passing in math (70 and greater)
passingMathCount = fullSchoolData[(fullSchoolData["math_score"] >=70)].count()["student_name"]
#print(passingMathCount)

passingMathPercentage = passingMathCount / float(studentCount) * 100
passingMathPercentage

74.9808526933878

In [131]:
# Calculate % passing in reading (70 and greater)
passingReadingCount = fullSchoolData[(fullSchoolData["reading_score"] >=70)].count()["student_name"]
#print(passingReadingCount)

passingReadingPercentage = passingReadingCount / float(studentCount) * 100
passingReadingPercentage

85.80546336482001

In [132]:
# Calculate % overall passing grades (both math and reading)
passingMathReadingCount = fullSchoolData[(fullSchoolData["math_score"] >=70) & (fullSchoolData["reading_score"] >=70)].count()["student_name"]

overallPassingRate = passingMathReadingCount / float(studentCount) *100
overallPassingRate

65.17232575950983

In [133]:
# Create and display a dataframe to hold of the above district summary metrics
district_summary = pd.DataFrame({"Total Schools": schoolCount,
    "Total Students":studentCount,"Total Budget": totalBudget,"Average Math Score":averageMathScore,
    "Average Reading Score": averageReadingScore, "% Passing in Math": passingMathPercentage,
    "% Passing in Reading": passingReadingPercentage, "% Overall Passing": overallPassingRate}, index=[0])
    
# Formatting
district_summary["Total Students"] = district_summary["Total Students"].map("{:,}".format)
district_summary["Total Budget"] = district_summary["Total Budget"].map("${:,.2f}".format)

# Display the DataFrame
district_summary

,Total Schools,Total Students,Total Budget,Average Math Score,Average Reading Score,% Passing in Math,% Passing in Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.980853,85.805463,65.172326


## School Summary

In [134]:
# School Name
schoolName_df = fullSchoolData.set_index('school_name').groupby(['school_name'])
schoolName_df

#School Type
schoolTypes= schoolData.set_index(['school_name'])['type']
#schoolTypes.head()

In [135]:
## Calcuate the total student count
perSchoolCounts = schoolName_df['Student ID'].count()
#perSchoolCounts.head()

In [136]:
# Calculate total school budget
perSchoolBudget = schoolName_df['budget'].first()
#perSchoolBudget.head()

In [137]:
# calculate budget per student per school
perSchoolCapita = perSchoolBudget / perSchoolCounts
#perSchoolCapita.head()

In [138]:
# Calculate the average math test score
perSchoolMath = schoolName_df["math_score"].mean()
#perSchoolMath.head()

In [139]:
# Calculate the average reading test score
perSchoolReading = schoolName_df["reading_score"].mean()
#perSchoolReading.head()

In [140]:
# Number of schools with passing math scores (70 or greater)
schoolPassingMath = fullSchoolData[fullSchoolData["math_score"] >= 70]
#schoolPassingMath.head()

In [141]:
# Calculate the number of schools with reading scores of 70 or higher
#schoolPassingReading = schoolData(schoolData["reading_score"] >=70).groupby('school_name')['Student ID'].count() / schoolStudentCount *100
schoolPassingReading = fullSchoolData[fullSchoolData["reading_score"] >= 70]
#schoolPassingReading.head()

In [142]:
# Use the provided code to calculate the schools that passed both math and reading with scores of 70 or higher
passingMathAndReading = fullSchoolData[(fullSchoolData["reading_score"] >= 70) & (fullSchoolData["math_score"] >= 70)]
#passingMathAndReading.head()

In [143]:
# Use the provided code to calculate the passing rates
perSchoolPassingMath = schoolPassingMath.groupby(["school_name"]).count()["student_name"] / perSchoolCounts * 100
perSchoolPassingReading = schoolPassingReading.groupby(["school_name"]).count()["student_name"] / perSchoolCounts * 100
overallPassingRate = passingMathAndReading.groupby(["school_name"]).count()["student_name"] / perSchoolCounts * 100

In [144]:
# Create a DataFrame called `per_school_summary` with columns for the calculations above.
per_school_summary= pd.DataFrame({"School Type": schoolTypes, "Total Students": perSchoolCounts, "Total School Budget": perSchoolBudget,
"Per Student Budget": perSchoolCapita, "Average Math Score": perSchoolMath, "Average Reading Score": perSchoolReading, 
"% Passing Math": perSchoolPassingMath, "% Passing Reading": perSchoolPassingReading, "% Overall Passing": overallPassingRate})

# Formatting
per_school_summary["Total School Budget"] = per_school_summary["Total School Budget"].map("${:,.2f}".format)
per_school_summary["Per Student Budget"] = per_school_summary["Per Student Budget"].map("${:,.2f}".format)

# Display the DataFrame
per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Highest-Performing Schools (by % Overall Passing)

In [145]:
# Sort the schools by `% Overall Passing` in descending order and display the top 5 rows. 
top_schools = per_school_summary.sort_values("% Overall Passing", ascending = False)
top_schools.head()

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272171,97.308869,90.948012
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.867718,96.539641,90.582567
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541


## Lowest-Perfoming Schools (by % Overall Passing)

In [146]:
# Sort the schools by `% Overall Passing` in ascending order and display the top 5 rows.
# Save the DataFrame as "bottom_schools"
bottom_schools = top_schools.tail().sort_values("% Overall Passing", ascending = True)
bottom_schools

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.366592,80.220055,52.988247
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172


## Math Scores by Grade

In [147]:
# Use the code provided to separate the data by grade
ninthGraders = fullSchoolData[(fullSchoolData["grade"] == "9th")]
tenthGraders = fullSchoolData[(fullSchoolData["grade"] == "10th")]
eleventhGraders = fullSchoolData[(fullSchoolData["grade"] == "11th")]
twelfthGraders = fullSchoolData[(fullSchoolData["grade"] == "12th")]

In [148]:
# Group by "school_name" and take the mean of each.
ninthGradersMathScores = ninthGraders.groupby(["school_name"]).mean()['math_score']
tenthGradersMathScores = tenthGraders.groupby(["school_name"]).mean()['math_score']
eleventhGradersMathScores = eleventhGraders.groupby(["school_name"]).mean()['math_score']
twelfthGradersMathScores = twelfthGraders.groupby(["school_name"]).mean()['math_score'] 

#print(ninthGradersMathScores.head())
#print(tenthGradersMathScores.head())
#print(eleventhGradersMathScores.head())
#print(twelfthGradersMathScores.head())

In [149]:
# Combine each of the scores above into single DataFrame called `math_scores_by_grade`
math_scores_by_grade = pd.DataFrame({"9th":ninthGradersMathScores, "10th":tenthGradersMathScores, 
"11th": eleventhGradersMathScores, "12th":twelfthGradersMathScores})

In [150]:
# Minor data wrangling
math_scores_by_grade.index.name = None

In [151]:
# Display the DataFrame
math_scores_by_grade

,9th,10th,11th,12th
Bailey High School,77.083676,76.996772,77.515588,76.492218
Cabrera High School,83.094697,83.154506,82.765560,83.277487
Figueroa High School,76.403037,76.539974,76.884344,77.151369
Ford High School,77.361345,77.672316,76.918058,76.179963
Griffin High School,82.044010,84.229064,83.842105,83.356164
Hernandez High School,77.438495,77.337408,77.136029,77.186567
Holden High School,83.787402,83.429825,85.000000,82.855422
Huang High School,77.027251,75.908735,76.446602,77.225641
Johnson High School,77.187857,76.691117,77.491653,76.863248
Pena High School,83.625455,83.372000,84.328125,84.121547


## Reading Scores by Grade

In [152]:
# Use the code provided to separate the data by grade
nintGraders = fullSchoolData[(fullSchoolData["grade"] == "9th")]
tenthGraders = fullSchoolData[(fullSchoolData["grade"] == "10th")]
eleventhGraders = fullSchoolData[(fullSchoolData["grade"] == "11th")]
twelfthGraders = fullSchoolData[(fullSchoolData["grade"] == "12th")]

In [153]:
# Group by "school_name" and take the mean of each.
ninthGradersReadingScore = ninthGraders.groupby(["school_name"]).mean()['reading_score']
tenthGradersReadingScores = tenthGraders.groupby(["school_name"]).mean()['reading_score']
eleventhGradersReadingScores = eleventhGraders.groupby(["school_name"]).mean()['reading_score']
twelfthGradersReadingScores = twelfthGraders.groupby(["school_name"]).mean()['reading_score'] 

In [154]:
# Combine each of the scores above into single DataFrame called `reading_scores_by_grade`
reading_scores_by_grade = pd.DataFrame({"9th":ninthGradersReadingScore, "10th":tenthGradersReadingScores, 
"11th": eleventhGradersReadingScores,"12th": twelfthGradersReadingScores})

In [155]:
# Minor data wrangling
reading_scores_by_grade = reading_scores_by_grade[["9th", "10th", "11th", "12th"]]
reading_scores_by_grade.index.name = None

In [156]:
# Display the DataFrame
reading_scores_by_grade

,9th,10th,11th,12th
Bailey High School,81.303155,80.907183,80.945643,80.912451
Cabrera High School,83.676136,84.253219,83.788382,84.287958
Figueroa High School,81.198598,81.408912,80.640339,81.384863
Ford High School,80.632653,81.262712,80.403642,80.662338
Griffin High School,83.369193,83.706897,84.288089,84.013699
Hernandez High School,80.866860,80.660147,81.396140,80.857143
Holden High School,83.677165,83.324561,83.815534,84.698795
Huang High School,81.290284,81.512386,81.417476,80.305983
Johnson High School,81.260714,80.773431,80.616027,81.227564
Pena High School,83.807273,83.612000,84.335938,84.591160


## Scores by School Spending

In [157]:
# Establish the bins 
spending_bins = [0, 585, 630, 645, 680]
labels = ["<$585", "$585-630", "$630-645", "$645-680"]

In [158]:
# Create a copy of the school summary since it has the "Per Student Budget" 
school_spending_df= per_school_summary.copy()

In [159]:
# Use `pd.cut` to categorize spending based on the bins.
#school_spending_df["Spending Ranges (Per Student)"] = pd.cut(school_spending_df["Per Student Budget"], spending_bins, labels =labels, include_lowest=True)
school_spending_df["Spending Ranges (Per Student)"] = pd.cut(perSchoolCapita, spending_bins, labels =labels, include_lowest=True)
school_spending_df

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,Spending Ranges (Per Student)
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283,$585-630
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769,<$585
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476,$630-645
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887,$630-645
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455,$585-630
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508,$645-680
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166,<$585
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884,$645-680
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172,$645-680
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541,$585-630


In [160]:
#  Calculate averages for the desired columns. 
spending_math_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Math Score"]
spending_reading_scores = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["Average Reading Score"]
spending_passing_math = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Math"]
spending_passing_reading = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Passing Reading"]
overall_passing_spending = school_spending_df.groupby(["Spending Ranges (Per Student)"]).mean()["% Overall Passing"]

In [161]:
# Assemble into DataFrame
spending_summary = pd.DataFrame({"Average Math Score" : spending_math_scores, "Average Reading Score": spending_reading_scores, 
"% Passing Math" : spending_passing_math, "% Passing Reading" : spending_passing_reading, "% Overall Passing" : overall_passing_spending})

# Display results
spending_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
Spending Ranges (Per Student),,,,,
<$585,83.455399,83.933814,93.460096,96.610877,90.369459
$585-630,81.899826,83.155286,87.133538,92.718205,81.418596
$630-645,78.518855,81.624473,73.484209,84.391793,62.857656
$645-680,76.997210,81.027843,66.164813,81.133951,53.526855


## Scores by School Size

In [162]:
# Establish the bins.
size_bins = [0, 1000, 2000, 5000]
labels = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [163]:
# Categorize the spending based on the bins
# Use `pd.cut` on the "Total Students" column of the `per_school_summary` DataFrame.
per_school_summary_df = per_school_summary.copy()
per_school_summary["School Size"] = pd.cut(per_school_summary_df["Total Students"], size_bins, labels = labels, include_lowest=True)
per_school_summary

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing,School Size
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680064,81.933280,54.642283,Large (2000-5000)
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133477,97.039828,91.334769,Medium (1000-2000)
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988471,80.739234,53.204476,Large (2000-5000)
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.309602,79.299014,54.289887,Large (2000-5000)
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392371,97.138965,90.599455,Medium (1000-2000)
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.752967,80.862999,53.527508,Large (2000-5000)
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.505855,96.252927,89.227166,Small (<1000)
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.683922,81.316421,53.513884,Large (2000-5000)
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.057551,81.222432,53.539172,Large (2000-5000)
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.594595,95.945946,90.540541,Small (<1000)


In [164]:
# Calculate averages for the desired columns. 
size_math_scores = per_school_summary.groupby(["School Size"]).mean()["Average Math Score"]
size_reading_scores = per_school_summary.groupby(["School Size"]).mean()["Average Reading Score"]
size_passing_math = per_school_summary.groupby(["School Size"]).mean()["% Passing Math"]
size_passing_reading = per_school_summary.groupby(["School Size"]).mean()["% Passing Reading"]
size_overall_passing = per_school_summary.groupby(["School Size"]).mean()["% Overall Passing"]

In [165]:
# Create a DataFrame called `size_summary` that breaks down school performance based on school size (small, medium, or large).
# Use the scores above to create a new DataFrame called `size_summary`
size_summary = pd.DataFrame({"Average Math Score" : size_math_scores, "Average Reading Score": size_reading_scores, 
"% Passing Math" : size_passing_math, "% Passing Reading" : size_passing_reading, "% Overall Passing" : size_overall_passing})

# Display results
size_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Size,,,,,
Small (<1000),83.821598,83.929843,93.550225,96.099437,89.883853
Medium (1000-2000),83.374684,83.864438,93.599695,96.790680,90.621535
Large (2000-5000),77.746417,81.344493,69.963361,82.766634,58.286003


## Scores by School Type

In [166]:
# Group the per_school_summary DataFrame by "School Type" and average the results.
type_math_scores = per_school_summary.groupby(["School Type"]).mean()[["Average Math Score"]]
type_reading_scores = per_school_summary.groupby(["School Type"]).mean()[["Average Reading Score"]]
type_passing_math = per_school_summary.groupby(["School Type"]).mean()[["% Passing Math"]]
type_passing_reading = per_school_summary.groupby(["School Type"]).mean()[["% Passing Reading"]]
type_overall_passing = per_school_summary.groupby(["School Type"]).mean()[["% Overall Passing"]]

In [167]:
# Use the code provided to select new column data
average_math_score_by_type = type_math_scores["Average Math Score"]
average_reading_score_by_type = type_reading_scores["Average Reading Score"]
average_percent_passing_math_by_type = type_passing_math["% Passing Math"]
average_percent_passing_reading_by_type = type_passing_reading["% Passing Reading"]
average_percent_overall_passing_by_type = type_overall_passing["% Overall Passing"]

In [168]:
# Assemble the new data by type into a DataFrame called `type_summary`
type_summary= pd.DataFrame({"Average Math Score" : average_math_score_by_type, "Average Reading Score": average_reading_score_by_type, 
"% Passing Math" : average_percent_passing_math_by_type, "% Passing Reading" : average_percent_passing_reading_by_type, 
"% Overall Passing" : average_percent_overall_passing_by_type})

# Display results
type_summary

,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
School Type,,,,,
Charter,83.473852,83.896421,93.620830,96.586489,90.432244
District,76.956733,80.966636,66.548453,80.799062,53.672208
